In [1]:
import requests
from pathlib import Path


In [2]:
BASE_URL = "https://spuservices.spu.gestao.gov.br"

API_KEY_OK = "98a7b6c5-d4e3-f2a1-b098-7c6d5e4f3a2b"
API_KEY_INVALIDA = "98a7b6c5-d4e3-f2a1-b098-7c6d5e4f3a2c"

BUCKET = "arquivos-acervo"
ARQUIVO_EXISTENTE = "1550846346565-null-8.tif-paraPDF.pdf"
ARQUIVO_INEXISTENTE = "arquivo_inexistente.pdf"

DOWNLOAD_DIR = Path("downloads")
DOWNLOAD_DIR.mkdir(exist_ok=True)


In [3]:
def baixar_arquivo(api_key, bucket, arquivo, nome_saida):
    url = f"{BASE_URL}/acervo/arquivo/{bucket}/{arquivo}"
    headers = {
        "X-API-Key": api_key
    }

    print(f"\n➡️ URL: {url}")
    print(f"🔑 API Key: {api_key}")

    response = requests.get(url, headers=headers, stream=True)

    print(f"📡 Status HTTP: {response.status_code}")
    print(f"📄 Content-Type: {response.headers.get('Content-Type')}")
    print(f"📦 Content-Length: {response.headers.get('Content-Length')}")

    if response.status_code == 200:
        caminho = DOWNLOAD_DIR / nome_saida
        with open(caminho, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)

        print(f"✅ Download concluído: {caminho}")
        print(f"📏 Tamanho (bytes): {caminho.stat().st_size}")

    else:
        print("❌ Download NÃO realizado")
        try:
            print("📨 Resposta:", response.json())
        except Exception:
            print("📨 Resposta (texto):", response.text)


In [4]:
baixar_arquivo(
    api_key=API_KEY_OK,
    bucket=BUCKET,
    arquivo=ARQUIVO_EXISTENTE,
    nome_saida="ok_arquivo_existente.pdf"
)


➡️ URL: https://spuservices.spu.gestao.gov.br/acervo/arquivo/arquivos-acervo/1550846346565-null-8.tif-paraPDF.pdf
🔑 API Key: 98a7b6c5-d4e3-f2a1-b098-7c6d5e4f3a2b
📡 Status HTTP: 200
📄 Content-Type: binary/octet-stream
📦 Content-Length: 4688757
✅ Download concluído: downloads/ok_arquivo_existente.pdf
📏 Tamanho (bytes): 4688757


In [5]:
baixar_arquivo(
    api_key=API_KEY_OK,
    bucket=BUCKET,
    arquivo=ARQUIVO_INEXISTENTE,
    nome_saida="nao_deveria_baixar.pdf"
)


➡️ URL: https://spuservices.spu.gestao.gov.br/acervo/arquivo/arquivos-acervo/arquivo_inexistente.pdf
🔑 API Key: 98a7b6c5-d4e3-f2a1-b098-7c6d5e4f3a2b
📡 Status HTTP: 404
📄 Content-Type: application/json
📦 Content-Length: 40
❌ Download NÃO realizado
📨 Resposta: {'error': 'Arquivo não encontrado'}


In [6]:
baixar_arquivo(
    api_key=API_KEY_INVALIDA,
    bucket=BUCKET,
    arquivo=ARQUIVO_EXISTENTE,
    nome_saida="nao_deveria_baixar.pdf"
)


➡️ URL: https://spuservices.spu.gestao.gov.br/acervo/arquivo/arquivos-acervo/1550846346565-null-8.tif-paraPDF.pdf
🔑 API Key: 98a7b6c5-d4e3-f2a1-b098-7c6d5e4f3a2c
📡 Status HTTP: 401
📄 Content-Type: application/json
📦 Content-Length: 61
❌ Download NÃO realizado
📨 Resposta: {'error': 'Chave de API inválida ou não fornecida'}
